In [1]:
import polars as pl
import polars.selectors as cs
from utils import PathsData, import_data, split_data, plot_correlation
from features import FeaturesFrame
from models import Model, ModelEnum, ParamGridEnum

import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from xgboost import XGBRegressor


x_validation = import_data(PathsData.X_TEST.value) #! to use in the end to make a submission
x = import_data(PathsData.X_TRAIN.value)
y = import_data(PathsData.Y_TRAIN.value)

x_train, x_test, y_train, y_test = split_data(x=x, y=y)

In [6]:
test_train = FeaturesFrame(x_train).encode_label(encoder='frequency').select('arret', 'encoded_gare')#.add_feature_square().scale_standard() #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="train")
test_test = FeaturesFrame(x_test).encode_label(encoder='frequency').select('arret', 'encoded_gare') #.add_feature_square().scale_standard(set='test', train_scaler=scaler) #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="test", train_scaler=scaler)

In [7]:
model = Model(model_enum=ModelEnum.XGBoost)
model_fit = model.fit(test_train.to_numpy(), y_train=y_train.to_numpy())
model.evaluate(X_test=test_test.to_numpy(), y_test=y_test.to_numpy())

Model Evaluation - MSE: 0.7891, R2: 0.0694, MAE: 0.6831


(0.7890712182688476, 0.06940980897448001, 0.6830813292153596)

In [12]:
model = Model(model_enum=ModelEnum.XGBoost)
params_opti_mlp = model.grid_search(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.XGBoost.value)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


In [8]:
params_opti_xgb = {
        "n_estimators": 200,
        "learning_rate": 0.1,
        "max_depth":5,
        "subsample": 0.8,
        "colsample_bytree": 1
}
    
opti_params = {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500, 'subsample': 0.8}
par = {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.8}
par_simple = {    "n_estimators":1250,   
    "max_depth":12,        
    "learning_rate":0.01,  
    "subsample":0.9,      
    "colsample_bytree":0.8,  
    "random_state":42,
    "n_jobs":-1}

In [9]:
mod = XGBRegressor().set_params(**par_simple)
mod_fit = mod.fit(test_train.to_numpy(),y_train.to_numpy())
preds = mod.predict(X=test_test.to_numpy())

In [10]:
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = mean_absolute_error(y_test,preds)
print(f"Model Evaluation - MSE: {mse:.4f}, R2: {r2:.4f}, MAE: {mae:.4f}")

Model Evaluation - MSE: 0.8205, R2: 0.0324, MAE: 0.7116


In [22]:
predictions_validation = mod.predict(FeaturesFrame(x_validation).encode_label(encoder='frequency').drop('train', 'gare', 'date').to_numpy())

In [23]:
pl.DataFrame(predictions_validation).rename({"column_0":"y_test"}).with_row_index().write_csv(r"C:\Users\faune\challenge-data-203\submissions\submission5.csv", separator=",")

In [ ]:
# pl.DataFrame(mod.predict(FeaturesFrame(x_validation).encode_label(encoder='frequency').select('arret', 'encoded_gare').to_numpy())).to_pandas().to_csv(r"C:\Users\faune\challenge-data-203\submissions\submission2.csv")

In [12]:
pl.DataFrame(predictions_validation).rename({"column_0":"y_test"}).with_row_index()

index,y_test
u32,f32
0,-0.461851
1,0.19992
2,0.32767
3,-0.251354
4,-0.654542
…,…
20652,0.013936
20653,0.085909
20654,-0.827888


In [ ]:
pl.read_csv(PathsData.X_TEST.value)